## Tolman–Oppenheimer–Volkoff equation Solver



### TOV equation

The TOV equation are use to describe the state parameters whose evolution equation are given by:


\begin{equation}
\frac{dp}{dr} = -(\epsilon + p)\frac{m\ +\ 4\pi r^3 p}{r(r\ -\ 2m)} \\
\frac{dm}{dr}=4 \pi r^2 \epsilon
\end{equation}

The above equation uses p, $\epsilon$ and $\rho$ in geometric units (Solar Mass). To convert the geometric units to $g/cm^3$ refer to the cell of Units.

### Generalised Piecewise Polytropes

The Generalised Piecewise Polytropes or GPP are the equations which gives the relation of the state parameters of density pressure and energy density.

The formalism for GPP followed in O_Boyle_2020 is:

Pressure and Dnsity:
\begin{equation}
p\ =\ K\rho^\Gamma\ +\ \Lambda
\end{equation}

Energy density and Density:
\begin{equation}
\epsilon\ =\ \frac{K}{\Gamma\ -\ 1}\rho^\Gamma\ +\ (1\ +\ a)\rho\ -\ \Lambda
\end{equation}


Density and $\eta$:
\begin{equation}
\rho\ =\ \big( \frac{\eta\ -\ a}{K(1\ +\ n)}\big)^n
\end{equation}

Pressure and $\eta$:
\begin{equation}
p\ =\ K\big(\frac{\eta\ -\ a}{K(1\ +\ n)}\big)^{1+n}\ +\ \Lambda
\end{equation}
Energy density and $\eta$:
\begin{equation}
\epsilon\ =\ \rho(\eta)\big(1\ +\ \frac{\eta n\ +\ a}{1\ +\ n} \big)\ -\ \Lambda
\end{equation}
where $\eta\ =\ h\ -\ 1$ and h is the enthalpy, $n\ =\ \frac{1}{\Gamma\ -\ 1}$, K is the polytropic constant, $\Lambda$ and a are Integration constants.

In [2]:
import numpy as np
from astropy.io import ascii
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.integrate import RK45
from scipy.integrate import solve_ivp
from scipy.integrate import odeint
from astropy.table import Table
#See last for only defining K_0

In [3]:
mev=1.60218*10**-13
kb=1.38065*10**-23
T=mev/kb
print(T)

11604534096.25901


### Equation of State

The parameters equation of state described Table 1 of the paper O_Boyle_2020 is used in the program to plot the Mass Radius relation. Under this section the user has to mention the EOS to be studied. A list of EOS which are available is given in the cell below.

### Units

Here the conversion from [$\mathrm{g\ cm^{-3}}$] to [$M_\odot$] is described. The variable **mul_fac** give the conversion value to be multiplied by the units of [$\mathrm{g\ cm^{-3}}$] to obtatined the quatity in [$M_\odot$].

Note: The units of pressure and energy density is also considered in $\mathrm{g/cm^3}$ as it was used in the paper O_Boyle_2020.

In [4]:
#Units to be converted at the end

G=6.6743*10**-11 #In SI 
c=2.9979*10**8 #In SI
km_m=10**3 
sol_mass=1.989*10**30 #In kg, SI


#Converting M_0 to km
km_M0= km_m * c**2 /(G * sol_mass)

#Multiplication factor to convert from M_0 to km
M0_km=1/km_M0
print(M0_km)

#Multiplication factor to convert from cgs to geometric units
mul_fac=(G/c**2)*(10**9*M0_km**2)
print(mul_fac)

1.477087498725341
1.6202569180215728e-18


### Definition of Pressure, Density and Energy Desnity

For all the functions first the peice index j is matched where the value of the given paramter lies for the Generalised Piecewise Polytrope. After the index j is obtained the parameters K, $\Gamma$, $\Lambda$ and a are used for that piece.

Note: Here $\Lambda$ is an integration constant and not the dimesnionless tidal deformability

1. Pressure as a function of $\eta$:

    Here $\eta$ is the variable hence the peice is selected with respect to $\eta$:

    \begin{equation}
    p\ =\ K_j \big[ \frac{\eta\ -\ a_j}{K_j\ (1\ +\ n_j) } \big]^{1\ +\ n_j} + \Lambda_j
    \end{equation}
    
2. $\eta$ as a function of Pressure:

    Here pressure (p) is the variable hence the peice is selected with respect to p:
    
    \begin{equation}
    \eta\ =\ K_j(1\ +\ n_j)\ \big[\frac{p\ -\ \Lambda_j}{K_j}\big]^{\frac{1}{1\ +\ n_j}} \ +\ a_j
    \end{equation}
3. Pressure as a function of density:

    Here desnity ($\rho$) is the variable hence the peice is selected with respect to $\rho$:
    
    \begin{equation}
    p=K_j \rho^{\Gamma_j} + \Lambda_j
    \end{equation}
    
4. Density as a funtion of $\eta$:

    Here $\eta$ is the variable hence the peice is selected with respect to $\eta$:
    
    \begin{equation}
    \rho\ =\ \big[ \frac{\eta\ -\ a_j}{K_j(1\ +\ n_j)} \big]^n_j
    \end{equation}
    
5. Energy density as a function of pressure:

    Here pressure (p) is the variable hence the peice is selected with respect to p:
    
    \begin{equation}
    \epsilon\ =\ \frac{p\ -\ \Lambda_j}{\Gamma_j\ -\ 1}\ +\ (1\ +\ a_j)\big(\frac{p\ -\ \Lambda_j}{K_j}\big)^{\frac{1}{\Gamma_j}}\ -\ \Lambda_j
    \end{equation}
    
6. Energy density as a function of $\eta$:

    Here $\eta$ is the variable hence the peice is selected with respect to $\eta$:
    
    \begin{equation}
    \epsilon\ =\ \rho(\eta)\big( 1\ +\ \frac{n_j\eta\ +\ a_j}{1\ +\ n_j} \big) - \Lambda_j
    \end{equation}
    
7. Energy density as a function of density:

    Here desnity ($\rho$) is the variable hence the peice is selected with respect to $\rho$:
    
    \begin{equation}
    \epsilon\ =\ \frac{K_j}{\Gamma_j\ -\ 1}\rho^\Gamma_j\ +\ (1\ +\ a_j)\rho\ -\ \Lambda_j
    \end{equation}
    

In [5]:
#functions of p rho e and eta


def p_eta(eta):
    
    if eta > eta_th[len(eta_th)-1]:
        j=len(eta_th)-1
    else:
        j=np.argmax(eta<eta_th) - 1
    
    pres= K[j]*((eta - a[j])/(K[j]*(1. + n[j])))**(1. + n[j]) + lam[j]
    #print(j)
    return pres
    

def eta_p(pres):
    
    if pres > p_th[len(p_th)-1]:
        j=len(p_th)-1
    else:
        j=np.argmax([pres<=p_th]) - 1
    
    eta = (((pres - lam[j])/K[j])**(1/(n[j]+1)))*K[j]*(n[j]+1) + a[j]
    
    return eta

def p_rho(rho):
    
    if rho > rho_th[len(rho_th)-1]:
        j=len(rho_th)-1
    else:
        j=np.argmax(rho<rho_th) - 1
    
    pres= K[j]*rho**gma[j] + lam[j]
    
    #print(j)
    return pres
    
def rho_eta(eta):
    
    if eta > eta_th[len(eta_th)-1]:
        j=len(eta_th)-1
    else:
        j=np.argmax(eta<=eta_th) - 1
    
    rho=((eta - a[j])/(K[j]*(1. + n[j])))**(n[j])
    #print(j)
    return rho
    
    
def e_pres(pres):
    
    if pres > p_th[len(p_th)-1]:
        j=len(p_th)-1
    else:
        j=np.argmax([pres<p_th]) - 1
    
    edes=((pres - lam[j])/(gma[j]-1)) + (1. + a[j])*((pres - lam[j])/K[j])**(1./gma[j]) - lam[j]
    
    return edes
    
    
def e_eta(eta):
    
    if eta > eta_th[len(eta_th)-1]:
        j=len(eta_th)-1
    else:
        j=np.argmax(eta<=eta_th) - 1
    
    
    eds=rho_eta(eta)*(1. + ((n[j]*eta + a[j])/(1+n[j]))) - lam[j]
    #print(j)
    return eds
    
    
def e_rho(rho):
    
    
    if rho > rho_th[len(rho_th)-1]:
        j=len(rho_th)-1
    else:
        j=np.argmax([rho<=rho_th]) - 1
        
    
    eds= (K[j]/(gma[j]-1.))*rho**gma[j] + (1+a[j])*rho - lam[j]
    #print((K[j]/(gma[j]-1.))*rho**gma[j],(1+a[j])*rho,lam[j])
    #print(j)
    #print(eds)
    return eds
    
def rho_pres(pres):
    
    if pres > p_th[len(p_th)-1]:
        j=len(p_th)-1
    else:
        j=np.argmax([pres<p_th]) - 1
    
    rho = ((pres-lam[j])/K[j])**(1./gma[j])
    
    return rho

def drho_dp(pres):
    
    if pres > p_th[len(p_th)-1]:
        j=len(p_th)-1
    else:
        j=np.argmax([pres<p_th]) - 1
    
    drho=(1./(K[j]*gma[j]))*((pres - lam[j])/K[j])**((1./gma[j]) - 1.)
    
    return drho


def de_dp(rho):
    
     
    if rho > rho_th[len(p_th)-1]:
        j=len(rho_th)-1
    else:
        j=np.argmax([rho<rho_th]) - 1
    
    de=(1./(gma[j]-1)) + ((1.+a[j])*(1./(K[j]*gma[j]))*rho**(1.-gma[j]))
    
    
    return de


def cs2_inv_pres(pres):
    
    if pres > p_th[len(p_th)-1]:
        j=len(p_th)-1
    else:
        j=np.argmax([pres<p_th]) - 1
        
    cs2_inv=(1./(gma[j] - 1.)) + ((1.+a[j])/(K[j]*gma[j]))*(pres/K[j])**((1.-gma[j])/gma[j])
    
    return cs2_inv


def cs2_rho(rho):
    
    if rho > rho_th[len(p_th)-1]:
        j=len(rho_th)-1
    else:
        j=np.argmax([rho<rho_th]) - 1
        
    cs2 = (gma[j]*K[j]*rho**(gma[j]-1.))/(((K[j]*gma[j]*rho**(gma[j]-1.))/(gma[j] - 1.)) + (1. + a[j]))
    
    return cs2
    
    
def df_dx(x,func):
    
    h=x*1.0e-7
    
    df=(-func(x+2.*h) + 8.*func(x+h) - 8.*func(x-h) + func(x-2.*h))/(12.*h)
    
    return df

def espl_rho(rho):
    
    if rho > rho_th[len(p_th)-1]:
        j=len(rho_th)-1
    else:
        j=np.argmax([rho<rho_th]) - 1
        
    espl=((K[j]*rho**(gma[j] - 1.))/(gma[j]-1.)) + a[j] - (lam[j]/rho)
    
    return espl
    
    
def dp_despl(rho):
    
    if rho > rho_th[len(p_th)-1]:
        j=len(rho_th)-1
    else:
        j=np.argmax([rho<rho_th]) - 1
        
    dp=(K[j] * gma[j] * rho**(gma[j] + 1.))/p_rho(rho)
    
    return dp
    
def dp_drho(rho):
    
    if rho > rho_th[len(p_th)-1]:
        j=len(rho_th)-1
    else:
        j=np.argmax([rho<rho_th]) - 1
        
    dp=K[j]*gma[j]*rho**(gma[j]-1.)
    
    return dp
    

### Parameters for GPP:

Here the parameters of the EOS are defined.

rho_th: dividing densities

p_th: pressure at the dividing densities

eta_th: $\eta$ at the dividing densities

K: Polytropic constant

gma: Adiabetic index $\Gamma$

lam: Integration constant $\Lambda$

a: Integration constant a

In [10]:
#Parameters of cold matter

#Enter the parameters
eos_used='SLY4'

#Compare to Exact EOS
#EOS of Sly4 and QHC19 are not available.
cmp=True
#Reading the eos
#This EOS is obtained from the Table 3 of O_Boyle_2020, DOI: https://arxiv.org/abs/2008.03342
eos=ascii.read('EOS_tables_ld.txt',names=['EOS','lgrho_0','lgK','g1','g2','g3'])

#Selecting the EOS to be used from the file EOS_tables.txt. 
#This files contains all the EOS from Table 3 of the paper O_Boyle_2020
eos_para=eos['EOS']==eos_used
eos_para=eos[eos_para]


#Using the values of dividning densities of Table 3 from the paper O_Boyle_2020 for the crust region and the core.
#The density joining the crust and the core is used according to the EOS table in use.
rho_th=np.array([0.0, 3.32e7, 1.0e8,3.35e11,5.317e11,0.0 ,10**14.87,10**14.99]) #10.0,2*10**8
#eos_para['lgrho_0'][0]
#Converting cgs to geometric units


#Defining K lambda gamma and a as zeros.
K=np.zeros(len(rho_th))
lam=np.zeros(len(rho_th))
gma=np.zeros(len(rho_th))
a=np.zeros(len(rho_th))

#Values setting the parameters
v1=5 #Index of the conjuction of crust and core
v2=v1+2

#The first 5 values of gamma according to Table 2 of the paper O_Boyle_2020.
gma[:v1]=[1.4791889792501332, 1.973, 1.269, -1.841,1.382]
#1.4329999999999492 1.9729999999998897

#The rest 3 values of gamma is taken from Table 3 of the paper O_Boyle_2020.
gma[v1:]=[eos_para['g1'],eos_para['g2'],eos_para['g3']]

#The first 5 values of K according to Table 2 of the paper O_Boyle_2020.
K[0:v1]=[3.0829909961101835e-08, 2.4943968895538545e-12, 1.662e-06, -7.957e+29,1.746e-8]
#The value of K to have continuity at the density where core and crust join.
#This value is obtatined from the Table 3 of O_Boyle_2020.
K[v1]=10**eos_para['lgK']


#Values of lambda and a from table 2 of Paper O_Boyle_2020.
#Values of the crust.
lam[0:v1]=[0.0, 2.390435454318849e3, -6.025e3, 1.193e9,7.077e8]
#Use this -587.5519383988526
#, -18147.29504674606

a[0:v1]=[0.0, 0.0002772079705228858, -0.0005278, 0.01035,8.208e-3]



#Converting cgs to geometric units
rho_th=rho_th*mul_fac
K=K*(mul_fac)**(1.-gma)
lam=lam*mul_fac

#getting the coonecting density at crust and core
rho_th[v1]=((K[v1]*gma[v1])/(K[v1-1]*gma[v1-1]))**(1./(gma[v1-1] - gma[v1])) 






#Calculating values of K for the pieces in the core given the condition of continuity and diffrentiability at the  dividing densities.
for i in range(v1,v2):
    K[i+1]= K[i] * (gma[i]/gma[i+1]) * rho_th[i+1]**(gma[i] - gma[i+1])


#Connecting the core and crust by imposing constraints on lambda.
lam[v1]=K[v1-1]*rho_th[v1]**gma[v1-1] + lam[v1-1] - K[v1]*rho_th[v1]**gma[v1]

#Calculating values of lambda for the pieces in the core given the condition of continuity and diffrentiability at the  dividing densities.
for i in range(v1,v2):
    lam[i+1]= lam[i] + (1. - (gma[i]/gma[i+1])) * K[i] * rho_th[i+1]**gma[i]



#Connecting the core and crust by imposing constraints on a.
a[v1]=(e_rho(rho_th[v1])/rho_th[v1]) - ((K[v1]/(gma[v1]-1.))*rho_th[v1]**(gma[v1] - 1.)) + (lam[v1]/rho_th[v1]) - 1.
#Calculating values of a for the pieces in the core given the condition of continuity and diffrentiability at the  dividing densities.
for i in range(v1,v2):
    a[i+1]= a[i] + (gma[i]*((gma[i+1] - gma[i])/((gma[i+1] - 1.)*(gma[i]-1.))))*K[i]*rho_th[i+1]**(gma[i]-1.)

#Calculaing n from given gamma
n=[1/(gma[i]-1.) for i in range(len(gma))]
n=np.array(n)
#Calculating the values of eta at the dividing densities.
eta_th=np.array([ ((rho_th[i]**(1./n[i]))*(K[i]*(1. + n[i]))) + a[i] for i in range(len(rho_th))])

#Calculating the values of pressure at the dividing densities.
p_th=np.array([K[j] * rho_th[j]**gma[j] + lam[j] for j in range(len(rho_th)) ])



In [11]:
#define TOV equation
def dpm_dr(r,y): 
    
    eta=eta_p(y[0])
    dpdr = - (e_pres(y[0]) + y[0])*(y[1] + 4.0*np.pi*r**3*y[0])/(r*(r-2.0*y[1]))
    dmdr = 4. * np.pi * r**2 * e_pres(y[0]) 
    dmbdr= 4. * np.pi * r**2. * (1. - (2.*y[1]/r))**(-0.5) * rho_pres(y[0])
   
    return np.array([dpdr,dmdr,dmbdr])

def dlm_dr(r,y,y_tdl):
    P=y[0]
    M=y[1]
    rho=rho_pres(P)
    F=(1. - (4.*np.pi*r**2)*(e_pres(P) - P))*(1. - (2.*M/r))**-1
    r2_Q=((4*np.pi*r**2)*(5.*e_pres(P) + 9.*P + (e_pres(P) + P)*(de_dp(rho)))*(1. - (2*M/r))**-1) - (6.*(1. - (2*M/r))**-1.) - ((4.*M**2/r**2)*(1. + (4.*np.pi*r**3*P/M))**2*(1.-(2.*M/r))**-2)
    dy_tdl_dr=-(y_tdl**2 + y_tdl*F + r2_Q)/r
    
    return dy_tdl_dr

### The fourth order Runge Kutta Method

In [12]:
def RK4(r,y,dr):
    
    y_temp =y
    
    k1     = dpm_dr(r,y)
    r,y    = adv_eq(r,y,k1,dr/2.)
    
    k2     = dpm_dr(r,y)
    r,y    = adv_eq(r,y,k2,dr/2.)
    
    k3     = dpm_dr(r, y)
    r,y    = adv_eq(r,y,k3,dr)
    
    k4     = dpm_dr(r,y)
    
    y_new  = y_temp +(dr/6.)*(k1 + 2.*k2 + 2.*k3 + k4)
    
    return y_new
    

def RK4_tdl(r,y,y_tdl,dr):
    
    y_temp_tdl =y_tdl
    
    k1_tdl=dlm_dr(r,y,y_tdl)
    r,y_tdl    = adv_eq_tdl(r,y_tdl,k1_tdl,dr/2.)
    
    k2_tdl=dlm_dr(r,y,y_tdl)
    r,y_tdl    = adv_eq_tdl(r,y_tdl,k2_tdl,dr/2.)
    
    k3_tdl=dlm_dr(r,y,y_tdl)
    r,y_tdl    = adv_eq_tdl(r,y_tdl,k3_tdl,dr)
    
    k4_tdl=dlm_dr(r,y,y_tdl)
    
    y_tdl_new=y_temp_tdl+(dr/6.)*(k1_tdl + 2.*k2_tdl + 2.*k3_tdl + k4_tdl)
    
    return y_tdl_new

    
    
def adv_eq(r,y,k,dr):
    
    #print(y_tdl,k_tdl)
    
    y = y + dr*k
    r = r + dr
    
    return r,y

def adv_eq_tdl(r,y_tdl,k_tdl,dr):
    
    #print(y_tdl,k_tdl)
    
    y_tdl=y_tdl+k_tdl*dr
    r = r + dr
    
    
    return r,y_tdl

### Computing mass and radius

The radius and mass are computed for either 50000 iterations or for pressure above $10^{-12} [\mathrm{M_\odot}]$ with the step size dr = 0.0005 [km].

The initial radius is taken as:
$r\ =\ 10^{-12}$

The initial mass is computed as:

\begin{equation}
m\ =\ \frac{4\pi r^3}{3} \epsilon(p)
\end{equation}

In [13]:
def compute_rm(rho_c_run):
    #central value and maximum number of steps
    pres        = p_rho(rho_c_run)
    pres_evl   = [pres]
    Nsteps = 50000 
    
    if pres > p_th[len(p_th)-1]:
        j=len(p_th)-1
    else:
        j=np.argmax([pres<=p_th]) - 1
    
    
    #initialize 
    r = 1e-12
    r_evl=[r]
    dr=0.0005
    m = (4./3.)* np.pi * r**3. *e_pres(pres)
    mb=0.
    m_evl=[m]
    i = 0 
    y_tdl=2.
    #print(pres)

    while ((i<Nsteps-2) & (pres >= 0.) ): #10.e-20  10.e-13
        #print(pres)
        y=np.array([pres,m,mb])
        y_tdl=RK4_tdl(r,y,y_tdl,dr)
        y=RK4(r,y,dr)
        #print(y)
        pres=y[0]
        pres_evl.append(pres)
        m=y[1]
        mb=y[2]
        m_evl.append(m)
        r=r+dr
        r_evl.append(r)
        i=i+1
        #print(m)
        
   
    
    C=y[1]/r
    y_new=y_tdl
    
    t1= 2. * C * (6. - 3. * y_new + 3. * C * (5.*y_new - 8.))
    t2= 4. * C**3 * (13. - 11. * y_new + C * (3. * y_new - 2.) + 2. * C**2 * (1. + y_new))
    t3= 3. * (1. - 2. * C)**2 * (2. - y_new + 2. * C *(y_new - 1.)) * np.log(1. - 2.* C)
    
    k2 = (8. * C**5 / 5.) * (1. - 2. * C)**2 * (2. + 2. * C * (y_new-1) - y_new) * (t1 + t2 + t3)**-1
    
    tidal=(2./3.)*k2*r**5
    dim_tidal=(2./3.)*k2*C**-5
    
    return(r,y[1],y[2],dim_tidal,k2)

### Main funtion that generates the values for mass and radius for a given central density.

The central density here is changed and the values of mass and radius is computed through which the Mass and Radius relation is obtained.

The in total 300 iterations for the central density starting from:
$\rho\ =\ 5.0e-4$

and is incremented with each iteration with the step size of $d\rho\ =\ 2.5e-5$


In [ ]:

#compute M vs R curve 


########################################### High Resolution ###############################
#iterations=600


#rad=[]
#mas=[]
#eta  = 0.03
#deta = 0.005

##############################################################################################


iterations=300



rad=[]
mas=[]
masb=[]
tid=[]
k_2=[]
rho_c=[]
#eta  = 0.03
#deta = 0.01

rho_c_run=5.0e-4
drho_c_run=2.5e-5


for i in range(iterations): #while (mass > mass_old): 

    #mass_old = mass

    (radius,mass,massb,tidal,k2) = compute_rm(rho_c_run)

    #eta= eta + deta
    print(i,rho_c_run/mul_fac,radius,mass,massb,tidal,k2)
    rho_c.append(rho_c_run)
    rad.append(radius)
    mas.append(mass)
    masb.append(massb)
    tid.append(tidal)
    k_2.append(k2)
    
    rho_c_run=rho_c_run+drho_c_run
    if i==0:
        continue
    else:
        if mas[i-1] > mas[i]:
            break

mas=np.array(mas)
rad=np.array(rad)*M0_km
masb=np.array(masb)


0 308593035116016.56 10.273500000002064 0.21493320708734107 0.21578094668219427 3565342.651125102 0.021434729251939812
1 324022686871817.4 9.792500000001477 0.23686424334952397 0.2382236366251971 2263431.1776039847 0.028111769166590948
2 339452338627618.2 9.425000000001027 0.2600935052444634 0.26207831171888735 1467167.487556843 0.0352218122125301
3 354881990383419.0 9.14000000000068 0.28454990331699115 0.2872855999520263 969444.4518131964 0.042528045292576755
4 370311642139219.8 8.916500000000406 0.3101623225847463 0.3137857952328251 652071.9942914253 0.049814977485038645
5 385741293895020.6 8.73950000000019 0.3368590379661971 0.3415181585706936 445937.60593524564 0.05690801582243138
6 401170945650821.4 8.597500000000016 0.36456747142602125 0.370420565348588 309735.60325495095 0.06369583659914722
7 416600597406622.2 8.483999999999877 0.39321420803444823 0.4004294178538006 218285.71636820256 0.07002599482898023
8 432030249162423.0 8.391499999999764 0.4227250670731781 0.4314796230627283

In [ ]:
ascii.write([rho_c,mas,rad,masb,tid,k_2],'tov_parik_%s_gpp_crust_new.dat'%eos_used,names=['rho_c','mass','radius','massb','tidal_dim_less','k2'])